In [1]:
number_of_shots = 3

In [2]:
experiment_name = "2023.12.06-04_Llama2_13B_API_3-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

In [9]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [10]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN


In [12]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [13]:
data_sel = data #.iloc[54:] #.sample(2)

In [24]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  file = open(log_filename, 'a+', encoding='utf-8')
    
  #print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)
  file.write(few_shots)
  file.write('\n')

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+str(result['choices'][0]['message']['content'].replace("\n"," - "))+"\n")
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR:"+str(e)+str(result)+"\n")
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_13B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1
  file.close()

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 11--- -  - The text is primarily about a subway train derailing in Brooklyn, New York, and the subsequent restoration of service. The content of the text does not seem to align with any of the other SDGs, so I have classified it as SDG 11, which focuses on sustainable cities and communities.
1 6099 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 16--- -  - The text does not directly relate to any specific SDG, but it does mention a celebrity's photo shoot in the Maldives, which is a popular tourist destination. SDG 16 focuses on peace, justice, and strong institutions, and the text does not seem to have any direct impact on these areas.


12 1524 0   ---SDG 13---
13 4660 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 3--- -  - The text is discussing the use of smartphones as thermometers to detect fevers, which is related to the goal of ensuring healthy lives and promoting well-being for all at all ages (SDG 3).
14 5077 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 0--- -  - The text provided does not seem to relate to any specific SDG, as it only mentions Google News and a news article. Therefore, the best classification is SDG 0, which represents "No specific SDG."
15 995 0   ---SDG 11---
16 2166 0  ---SDG 0---
17 2512 0   Sure, I'd be happy to help! Here's the text you provided, with the SDG label that best represents the content of the text enclosed in triple quotes: -  - 

32 3604 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 12--- -  - The text does not mention any specific SDGs, but it does mention "trade" and "market," which are related to SDG 12: Responsible Consumption and Production.
33 3940 0  <<SYS>> - ---SDG 16--- -  - The text describes a fatal crash and the death of a person, which falls under the category of "Peace, Justice and Strong Institutions" (SDG 16).
34 6228 0   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 4--- -  - The text discusses the importance of high school athletes being role models and building relationships with young children who look up to them, which aligns with SDG 4: Quality Education.
35 3148 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by 

52 6243 0   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16--- -  - The text discusses the issue of politics and the Uniform Civil Code (UCC), with a focus on the marginalized Pasmanda community and the actions of the BJP and other political parties. The text also touches on the topic of law and order in Delhi and the remarks of West Bengal Chief Minister Mamata Banerjee. Overall, the text is focused on political issues and the rights of marginalized communities, which aligns with SDG 16 (peace, justice and strong institutions).
53 4104 0   Sure! Here's the text you provided, with the label for each paragraph: -  - ---SDG 3--- -  - Jamie Foxx was rushed to a hospital from the sets of his upcoming action-comedy Back In Action in April due to a mysterious illness. Since then, the actor’s co-stars and fans have been showing their concern as there have been a few updates about Foxx’s health. On Sunday, Jamie’s co-star Porscha Coleman gave a major upda

69 6667 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 1--- -  - The text describes the high percentage of children living in extreme poverty, with severe deprivations in terms of dwellings, access to basic necessities like water and sanitation, and access to education and healthcare. This is in line with SDG 1, which aims to end poverty in all its forms, including extreme poverty, by 2030.
70 4588 1   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 1--- -  - The text focuses on the importance of access to land, agricultural inputs, finance, extension services, and markets for smallholder producers in rural areas, which is essential for enhancing food security and reducing poverty. The policies mentioned i

...

934 2084 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 15--- -  - The text describes a project that aims to halt land degradation and provide sustainable income opportunities for local communities, which aligns with SDG 15: Life On Land.
935 4203 15   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the importance of biodiversity and ecosystem services, and the need for effective frameworks and financing strategies to protect and conserve these resources. It also mentions the need for robust, comprehensive, and comparable data to track public biodiversity expenditure and assess the economic benefits of addressing environmental externalities. Therefore, the most appropriate SDG label for this text is SDG 15 (Life On La

952 389 15   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text you provided mentions "biodiversity" and "outdoors in Scotland," which suggests that it is related to the conservation and sustainable use of natural resources, specifically SDG 15: Life On Land.
953 943 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 15--- -  - The text describes the loss of biodiversity and the decline of species populations, which is directly related to SDG 15: Life On Land.
954 6522 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the importance of biodiversity and ecosystems for 

970 3501 16   ---SDG 16---
971 4998 16 ---SDG 16---
972 6738 16  ---SDG 16---
973 3540 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 16--- -  - The text discusses the issue of human rights violations in Eritrea and the potential for accountability for these crimes. It mentions the UN Commission of Inquiry on Human Rights in Eritrea (COI) and the principle of complementarity, which is relevant to SDG 16 (peace, justice, and strong institutions).
974 283 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 16--- -  - The text discusses the public enforcement regime in China and its limitations, particularly the lack of reliable private enforcement and the pattern of selective enforcement. This is best classified under SDG 16, which focuses on peaceful and inclusive societies, access to justice for all,

995 567 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 16--- -  - The text discusses the impact of government interventions on the internet, specifically in regards to surveillance, censorship, and the restriction of freedom of expression and opinion. These issues are closely related to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice for all, and effective and accountable institutions. The text highlights the abuse of jurisdictional power and attacks on human rights, particularly freedom of expression and opinion, which are essential for achieving SDG 16.
996 4214 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 16--- -  - The text focuses on the role of e-government policies in reducing corruption and promoting g

1012 2870 16   Sure, I'd be happy to help! Here's the text you provided, with my classification of each paragraph according to the Sustainable Development Goals (SDGs): -  - 1. What's morality got to do with it? making the right distinctions Jean Bethke Elshtain -----SDG 16---- - 2. Unauthorized humanitarian intervention Mark Stein -----SDG 16---- - 3. Thinking constitutionally: the problem of deliberative democracy Stephen L. Elkin -----SDG 16---- - 4. Representing ignorance Russell Hardin -----SDG 16---- - 5. Dual citizenship and American democracy: patriotism, national attachment, and national identity Stanley A. Renshon -----SDG 16---- - 6. Policy implications of zero discounting: an exploration in politics and morality Tyler Cowen -----SDG 16---- - 7. Reflections on espionage Harvey Klehr -----SDG 16---- - 8. Mr. Pinocchio goes to Washington: lying in politics Robert Weissberg -----SDG 16---- - 9. A subject of distaste an object of judgment John Haldane -----SDG 16---- - 10. Again

In [25]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [26]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(725, 5328, 12, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 12---\n\nThe text discusses recycling and waste management, specifically the impact of convenience factors on recycling volume. This aligns with SDG 12, which focuses on responsible consumption and production patterns.")
(396, 3056, 6, "  Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:\n\n---SDG 6---\n\nThe text describes improvements in access to drainage and wastewater treatment, with a significant increase in households connected to public sewerage systems and wastewater treatment plants. This aligns with SDG 6, which aims to ensure the availability and sustainable management of water and sanitation for all.")
(988, 2472, 16, "  Sure, I'd be happy to help! Here's the classification of the text according to 

In [27]:
len(tuple_list)

1020

In [28]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [29]:
results_processed

,cnt,id,true_label,predicted_label
0,725,5328,12,"Sure, I'd be happy to help! Here's the class..."
1,396,3056,6,"Sure, I'd be happy to help! Here's the class..."
2,988,2472,16,"Sure, I'd be happy to help! Here's the class..."
3,414,2416,6,"Sure, I'd be happy to help! Here's the class..."
4,802,6675,13,"Sure, I'd be happy to help! Here's the class..."
...,...,...,...,...
1015,357,747,5,"Sure, I'd be happy to help! Here's the class..."
1016,540,1607,9,"Sure, I'd be happy to help! Here's the class..."
1017,332,2861,5,"Sure, I'd be happy to help! Here's the class..."
1018,888,1089,14,"Sure, I'd be happy to help! Here's the class..."


In [30]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 0---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class..."
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class..."
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class..."
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class..."


In [31]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class...",16
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 0---,0
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",7
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class...",16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class...",16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class...",16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class...",16


In [32]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,11,0,11,11,0,0,0,11,11,11,11
1,86,85,86,86,85,85,85,86,86,86,86
10,42,39,42,42,39,39,39,42,42,42,42
11,70,64,70,70,64,64,64,70,70,70,70
12,74,66,74,74,66,66,66,74,74,74,74
13,50,47,50,50,47,47,47,50,50,50,50
14,57,55,57,57,55,55,55,57,57,57,57
15,62,60,62,62,60,60,60,62,62,62,62
16,71,66,71,71,66,66,66,71,71,71,71


In [33]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [34]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.18      0.31        60
           1       0.60      0.87      0.71        60
           2       0.88      0.58      0.70        60
           3       0.75      0.93      0.83        60
           4       0.66      0.98      0.79        60
           5       0.89      0.80      0.84        60
           6       0.96      0.75      0.84        60
           7       0.87      0.75      0.80        60
           8       0.51      0.60      0.55        60
           9       0.76      0.58      0.66        60
          10       0.64      0.45      0.53        60
          11       0.70      0.82      0.75        60
          12       0.66      0.82      0.73        60
          13       0.80      0.67      0.73        60
          14       0.91      0.87      0.89        60
          15       0.85      0.88      0.87        60
          16       0.77      0.92      0.84        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
